In [332]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [314]:
df = pd.read_csv("./input/temp.csv")
df.head()

,APP_NAME,RATING,CATEGORY,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,REVIEW_COUNT,...,DEVELOPER,DEVELOPER_ADDRESS,CONTENT_RATING,AD_SUPPORTED,COUNTRY,IN_APP_PURCHASES,EDITORS_CHOICE,DAYS_SINCE_UPDATE,DAYS_SINCE_RELEASED,SIZEBAND
0,"""Ghost Voice Catcher"" AUTO EVP",3.933333,Lifestyle,68,12,1,6,7,42,24.0,...,BIG BEARD Studios,114 1/2 Ash st. Nashua NH 03060,Everyone,False,Usa,False,False,1820.0,3482.0,very small
1,"""OXXO""",4.740000,Game,975,23,0,15,94,843,54.0,...,Hamster On Coke Games,ul Jugosłowiańska 37F/10\n73-110 Stargard Szcz...,Everyone,False,NaN,False,False,0.0,945.0,medium
2,#DRIVE,4.434152,Game,229679,9352,4740,15249,47804,152534,6473.0,...,Pixel Perfect Dude S.A.,ul. Polna 28/6 lok. 4\n62-800 Kalisz\nPolska,Everyone,True,NaN,True,False,4.0,1084.0,large
3,#SelfCare,4.463476,Game,14932,865,336,1088,1353,11290,1501.0,...,TRU LUV Inc.,"186-720 King Street West\nToronto, Ontario, Ca...",Teen,False,NaN,True,False,952.0,1275.0,medium
4,#open Polyamorous + ENM Dating,2.550000,Dating,708,263,163,63,70,149,362.0,...,#open,NaN,Mature 17+,False,NaN,True,False,17.0,1026.0,small


In [301]:
df["SIZEBAND"].value_counts()

medium        9613
small         4313
large         2473
very small     917
very large     153
Name: SIZEBAND, dtype: int64

In [302]:
df["PRICEBAND"] = "No Price Band"
df.loc[ df['PRICE'] == 0, 'PRICEBAND'] = 'Free'
df.loc[(df['PRICE'] > 0) & (df['PRICE'] <= 0.99), 'PRICEBAND'] = 'cheap'
df.loc[(df['PRICE'] > 0.99) & (df['PRICE'] <= 2.99), 'PRICEBAND']   = 'not cheap'
df.loc[(df['PRICE'] > 2.99) & (df['PRICE'] <= 4.99), 'PRICEBAND']   = 'normal'
df.loc[(df['PRICE'] > 4.99) & (df['PRICE'] <= 14.99), 'PRICEBAND']   = 'expensive'
df.loc[(df['PRICE'] > 14.99) & (df['PRICE'] <= 29.99), 'PRICEBAND']   = 'too expensive'
df.loc[(df['PRICE'] > 29.99), 'PRICEBAND']  = 'FXXXing expensive'
df["PRICEBAND"].value_counts()

Free                 13912
not cheap             1315
normal                 913
expensive              708
cheap                  480
too expensive          116
FXXXing expensive       25
Name: PRICEBAND, dtype: int64

In [303]:
df['INSTALLS_GROUP'] = 'More than 100M'
df.loc[(df['INSTALLS'] == '1+') | (df['INSTALLS'] == '10+') | (df['INSTALLS'] == '50+') | 
       (df['INSTALLS'] == '100+') | (df['INSTALLS'] == '500+' ) | (df['INSTALLS'] == '1,000+' ) | 
       (df['INSTALLS'] == '5,000+' ) | (df['INSTALLS'] == '10,000+' ),'INSTALLS_GROUP'] = 'Less than 10K'
df.loc[(df['INSTALLS'] == '50,000+') | (df['INSTALLS'] == '100,000+')  | (df['INSTALLS'] == '500,000+') 
       | (df['INSTALLS'] == '1,000,000+')  | (df['INSTALLS'] == '5,000,000+') ,'INSTALLS_GROUP'] = 'Between 10K and 1M'
df.loc[(df['INSTALLS'] == '5,000,000+') | (df['INSTALLS'] == '10,000,000+') | (df['INSTALLS'] == '50,000,000+') 
       | (df['INSTALLS'] == '100,000,000+'),'INSTALLS_GROUP'] = 'Between 1M and 100M'
df['INSTALLS_GROUP'] .value_counts()

Between 10K and 1M     8800
Between 1M and 100M    4636
Less than 10K          3930
More than 100M          103
Name: INSTALLS_GROUP, dtype: int64

In [304]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17469 entries, 0 to 17468
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   APP_NAME             17469 non-null  object 
 1   RATING               17469 non-null  float64
 2   CATEGORY             17469 non-null  object 
 3   RATING_COUNT         17469 non-null  int64  
 4   1_STAR_RATINGS       17469 non-null  int64  
 5   2_STAR_RATINGS       17469 non-null  int64  
 6   3_STAR_RATINGS       17469 non-null  int64  
 7   4_STAR_RATINGS       17469 non-null  int64  
 8   5_STAR_RATINGS       17469 non-null  int64  
 9   REVIEW_COUNT         17469 non-null  float64
 10  INSTALLS             17469 non-null  object 
 11  MAX_INSTALLS         17469 non-null  int64  
 12  FREE                 17469 non-null  bool   
 13  PRICE                17469 non-null  float64
 14  SIZE                 17469 non-null  float64
 15  DEVELOPER            17469 non-null 

---

## Model Building

First we identify the which features are usable in the dataset:
1. RATING
2. CATEGORY 
3. INSTALLS
4. SIZEBAND
5. FREE
6. PRICEBAND
7. AD_SUPPORTED
8. CONTENT_RATING
9. IN_APP_PURCHASES
10. EDITORS_CHOICE

Data preprocesssing (handle null values -> handle categorical data -

In [305]:
df["RATING_GROUP"] = round(df["RATING"], 0) 
for i in range(df.shape[0]):
    if (df.loc[i, "RATING_GROUP"] <= 3):
        df.loc[i, "RATING_GROUP"]  = 3
df["RATING_GROUP"].value_counts()

4.0    9858
5.0    5520
3.0    2091
Name: RATING_GROUP, dtype: int64

### Data preprocessing

In [307]:
feature_list = ["RATING_GROUP", "CATEGORY", "INSTALLS_GROUP", "SIZEBAND", "FREE", "PRICEBAND", "AD_SUPPORTED", "CONTENT_RATING", "IN_APP_PURCHASES", "EDITORS_CHOICE"]
model_df = df[feature_list].copy()
model_df.head()

,RATING_GROUP,CATEGORY,INSTALLS_GROUP,SIZEBAND,FREE,PRICEBAND,AD_SUPPORTED,CONTENT_RATING,IN_APP_PURCHASES,EDITORS_CHOICE
0,4.0,Lifestyle,Less than 10K,very small,False,expensive,False,Everyone,False,False
1,5.0,Game,Less than 10K,medium,False,cheap,False,Everyone,False,False
2,4.0,Game,Between 1M and 100M,large,True,Free,True,Everyone,True,False
3,4.0,Game,Between 10K and 1M,medium,True,Free,False,Teen,True,False
4,3.0,Dating,Between 10K and 1M,small,True,Free,False,Mature 17+,True,False


#### Handling null values

In [308]:
model_df.isna().sum()

RATING_GROUP        0
CATEGORY            0
INSTALLS_GROUP      0
SIZEBAND            0
FREE                0
PRICEBAND           0
AD_SUPPORTED        0
CONTENT_RATING      0
IN_APP_PURCHASES    0
EDITORS_CHOICE      0
dtype: int64

Seems like there are no null values in our dataset :)

#### Handling categorical values

Before we continue, our categorical variables are further divided into 2 types —
- Ordinal categorical variables - `INSTALLS_GROUP`, `SIZEBAND`, `PRICEBAND`
- Nominal categorical variable - `CATEGORY`, `FREE`, `AD_SUPPORTED`, `CONTENT_RATING`, `IN_APP_PURCHASES`, `EDITORS_CHOICE`

In [309]:
# numerical_features = ["DAYS_SINCE_RELEASED"]
nominal_cat = ["CATEGORY","FREE","AD_SUPPORTED", "IN_APP_PURCHASES", "CONTENT_RATING", "EDITORS_CHOICE"]
ordinal_cat = ["INSTALLS_GROUP", "SIZEBAND", "PRICEBAND"]

**Handling Ordinal Categorical Variables**

In [310]:
ig_mapping = {"Less than 10K": 0, "Between 10K and 1M": 1, "Between 1M and 100M": 2, "More than 100M": 3}
pb_mapping = {"Free": 0, "cheap": 1, "not cheap": 2,"normal": 3,"expensive": 4,"too expensive": 5, "FXXXing expensive": 6 } 
sb_mapping = {"very small": 0, "small": 1, "medium": 2, "large": 3, "very large": 4}

model_df["INSTALLS_GROUP"] = model_df["INSTALLS_GROUP"].map(ig_mapping)
model_df["PRICEBAND"] = model_df["PRICEBAND"].map(pb_mapping)
model_df["SIZEBAND"] = model_df["SIZEBAND"].map(sb_mapping)

model_df[["INSTALLS_GROUP", "PRICEBAND", "SIZEBAND"]].head()

,INSTALLS_GROUP,PRICEBAND,SIZEBAND
0,0,4,0
1,0,1,2
2,2,0,3
3,1,0,2
4,1,0,1


**Handling Nominal Categorical Variables**

In [311]:
# Import the encoder from sklearn
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()

# OneHotEncoding of categorical predictors 
model_df_cat = model_df[nominal_cat]
ohe.fit(model_df_cat)
model_df_cat_ohe = pd.DataFrame(ohe.transform(model_df_cat).toarray(), 
                                  columns=ohe.get_feature_names_out(model_df_cat.columns))

# Check the encoded variables
model_df_cat_ohe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17469 entries, 0 to 17468
Data columns (total 45 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   CATEGORY_Art & Design             17469 non-null  float64
 1   CATEGORY_Auto & Vehicles          17469 non-null  float64
 2   CATEGORY_Beauty                   17469 non-null  float64
 3   CATEGORY_Books & Reference        17469 non-null  float64
 4   CATEGORY_Business                 17469 non-null  float64
 5   CATEGORY_Comics                   17469 non-null  float64
 6   CATEGORY_Communication            17469 non-null  float64
 7   CATEGORY_Dating                   17469 non-null  float64
 8   CATEGORY_Education                17469 non-null  float64
 9   CATEGORY_Entertainment            17469 non-null  float64
 10  CATEGORY_Events                   17469 non-null  float64
 11  CATEGORY_Finance                  17469 non-null  float64
 12  CATE

In [312]:
model_df_ord = model_df[ordinal_cat]
model_df_ord.loc[200]

INSTALLS_GROUP    1
SIZEBAND          4
PRICEBAND         2
Name: 200, dtype: int64

In [315]:
# Combining Numeric features with the categorical features
# model_df_num = model_df[numerical_features]
model_df_res = model_df["RATING_GROUP"]
model_df_ord = model_df[ordinal_cat]
model_df_encoded = pd.concat([ model_df_cat_ohe, model_df_ord, model_df_res], 
                           sort = False, axis = 1).reindex(index=model_df_res.index)

# Check the final dataframe
model_df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17469 entries, 0 to 17468
Data columns (total 49 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   CATEGORY_Art & Design             17469 non-null  float64
 1   CATEGORY_Auto & Vehicles          17469 non-null  float64
 2   CATEGORY_Beauty                   17469 non-null  float64
 3   CATEGORY_Books & Reference        17469 non-null  float64
 4   CATEGORY_Business                 17469 non-null  float64
 5   CATEGORY_Comics                   17469 non-null  float64
 6   CATEGORY_Communication            17469 non-null  float64
 7   CATEGORY_Dating                   17469 non-null  float64
 8   CATEGORY_Education                17469 non-null  float64
 9   CATEGORY_Entertainment            17469 non-null  float64
 10  CATEGORY_Events                   17469 non-null  float64
 11  CATEGORY_Finance                  17469 non-null  float64
 12  CATE

### Machine learning

In [325]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix,ConfusionMatrixDisplay,classification_report
from sklearn.model_selection import GridSearchCV

In [317]:
def rmse_cv(model):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=5)).mean()
    return rmse
    

def evaluation(y, predictions):
    mae = mean_absolute_error(y, predictions)
    mse = mean_squared_error(y, predictions)
    rmse = np.sqrt(mean_squared_error(y, predictions))
    r_squared = r2_score(y, predictions)
    return mae, mse, rmse, r_squared

models = pd.DataFrame(columns=["Model","MAE","MSE","RMSE","R2 Score","RMSE (Cross-Validation)"])

In [318]:
# Extract Response and Predictors
y = pd.DataFrame(model_df_encoded["RATING_GROUP"])
X = pd.DataFrame(model_df_encoded.drop(["RATING_GROUP"], axis = 1))

# Split the Dataset into Train and Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=20)

X_train.isna().sum()

CATEGORY_Art & Design               0
CATEGORY_Auto & Vehicles            0
CATEGORY_Beauty                     0
CATEGORY_Books & Reference          0
CATEGORY_Business                   0
CATEGORY_Comics                     0
CATEGORY_Communication              0
CATEGORY_Dating                     0
CATEGORY_Education                  0
CATEGORY_Entertainment              0
CATEGORY_Events                     0
CATEGORY_Finance                    0
CATEGORY_Food & Drink               0
CATEGORY_Game                       0
CATEGORY_Health & Fitness           0
CATEGORY_House & Home               0
CATEGORY_Libraries & Demo           0
CATEGORY_Lifestyle                  0
CATEGORY_Maps & Navigation          0
CATEGORY_Medical                    0
CATEGORY_Music & Audio              0
CATEGORY_News & Magazines           0
CATEGORY_Parenting                  0
CATEGORY_Personalization            0
CATEGORY_Photography                0
CATEGORY_Productivity               0
CATEGORY_Sho

In [319]:
# Logistic Regression

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, y_train) * 100, 2)
acc_log

C:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


56.15

In [320]:
# Support Vector Machines

svc = SVC()
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, y_train) * 100, 2)
acc_svc

C:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


57.03

In [322]:
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

         3.0       0.00      0.00      0.00       418
         4.0       0.58      0.97      0.72      2005
         5.0       0.45      0.05      0.09      1071

    accuracy                           0.57      3494
   macro avg       0.34      0.34      0.27      3494
weighted avg       0.47      0.57      0.44      3494

[[   0  412    6]
 [   0 1942   63]
 [   0 1015   56]]


C:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [327]:
param_grid = {'C':[1,10,100,1000],'gamma':[1,0.1,0.001,0.0001], 'kernel':['linear','rbf']}

In [331]:
grid = GridSearchCV(SVC(),param_grid,refit = True, verbose=2)

In [333]:
grid.fit(X_train,y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] END ........................C=1, gamma=1, kernel=linear; total time=   8.5s
[CV] END ........................C=1, gamma=1, kernel=linear; total time=   8.5s
[CV] END ........................C=1, gamma=1, kernel=linear; total time=   8.4s
[CV] END ........................C=1, gamma=1, kernel=linear; total time=   8.2s
[CV] END ........................C=1, gamma=1, kernel=linear; total time=   8.2s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=  13.3s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=  13.8s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=  14.7s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=  13.8s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=  13.3s
[CV] END ......................C=1, gamma=0.1, kernel=linear; total time=   8.8s
[CV] END ......................C=1, gamma=0.1, 

GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.001, 0.0001],
                         'kernel': ['linear', 'rbf']},
             verbose=2)

In [336]:
grid.best_params_

{'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}

In [334]:
predic = grid.predict(X_test)

In [335]:
print(classification_report(y_test,predic))
print(confusion_matrix(y_test, predic))

              precision    recall  f1-score   support

         3.0       0.35      0.03      0.06       418
         4.0       0.59      0.91      0.72      2005
         5.0       0.49      0.17      0.26      1071

    accuracy                           0.58      3494
   macro avg       0.48      0.37      0.34      3494
weighted avg       0.53      0.58      0.50      3494

[[  13  376   29]
 [  17 1820  168]
 [   7  877  187]]


#### Predicting Installs

In [338]:
# Extract Response and Predictors
y = pd.DataFrame(model_df_encoded["INSTALLS_GROUP"])
X = pd.DataFrame(model_df_encoded.drop(["RATING_GROUP", "INSTALLS_GROUP"], axis = 1))

# Split the Dataset into Train and Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=20)

In [343]:
svc_installs = SVC()
svc_installs.fit(X_train, y_train)
prediction = svc_installs.predict(X_test)


In [344]:
print(classification_report(y_test,prediction))
print(confusion_matrix(y_test, prediction))

              precision    recall  f1-score   support

           0       0.72      0.59      0.65       794
           1       0.61      0.65      0.63      1711
           2       0.55      0.56      0.55       965
           3       0.00      0.00      0.00        24

    accuracy                           0.61      3494
   macro avg       0.47      0.45      0.46      3494
weighted avg       0.61      0.61      0.61      3494

[[ 470  296   28    0]
 [ 187 1112  412    0]
 [   0  420  545    0]
 [   0    9   15    0]]
